# Register SmartSPIM Data To CCF v3.1 Mouse Brain Atlas

## Overview

This notebook demonstrates a reproducible registration pipeline to align downsampled SmartSPIM data to the Allen Mouse Brain Common Coordinate Framework (CCF)$^1$. The notebook can be run interactively in CodeOcean to evaluate intermediate results or can be run from start to end for reproducible results in the CodeOcean capsule$^2$.

## Inputs

1. Target (fixed) image. The CCF v3.1 atlas with updated spacing and spatial orientation is available in NIFTI format at http://download.alleninstitute.org/informatics-archive/converted_mouse_ccf/average_template/. More information on the CCF atlas is available at http://help.brain-map.org/display/mouseconnectivity/API

2. Source (moving) image. Stitched SmartSPIM mouse brain images are available on the AWS S3 "aind-open-data" bucket in Zarr format. The largest resolution / smallest image size is used here for performance considerations.

## Outputs

1. Registered source (moving) image aligned to CCF atlas space.

2. ITK multistage composite transform mapping from source to target space.

## Assumptions

1. The source and target images are spatially oriented to common anatomical directions. This can be confirmed with a 3D spatial viewer such as 3D Slicer, ITKWidgets, or Neuroglancer. Note that viewers displaying voxel data without spatial information such as matplotlib may produce misleading visuals showing data aligned to different anatomical axes.

2. The source image contains correct metadata. In this notebook the Zarr input sample is updated with known spatial metadata.

## Procedure

1. Data is read in from their respective stores attached via CodeOcean's data attachment mechanism.

2. An initial translation is constructed to coarsely superimpose the source image on the target image. The source image is updated in place without resampling.

3. The source image is registered to the target image. ITKElastix is used to optimize three transform stages for rigid, affine, and then deformable registration.

4. Results are written out.

## References

1. Quanxin Wang, Song-Lin Ding, Yang Li, Josh Royall, David Feng, Phil Lesnar, Nile Graddis, Maitham Naeemi, Benjamin Facer, Anh Ho, Tim Dolbeare, Brandon Blanchard, Nick Dee, Wayne Wakeman, Karla E. Hirokawa, Aaron Szafer, Susan M. Sunkin, Seung Wook Oh, Amy Bernard, John W. Phillips, Michael Hawrylycz, Christof Koch, Hongkui Zeng, Julie A. Harris, Lydia Ng,
The Allen Mouse Brain Common Coordinate Framework: A 3D Reference Atlas, Cell, Volume 181, Issue 4, 2020, Pages 936-953.e20, ISSN 0092-8674, https://doi.org/10.1016/j.cell.2020.04.007

2. https://help.codeocean.com/en/articles/1111013-rendering-jupyter-notebooks-to-html

## Initialize Notebook

In [ ]:
import os
import itertools

import itk
import itkwidgets
import numpy as np

assert "ElastixRegistrationMethod" in dir(
    itk
)  # Ensure itk-elastix is installed

itk.auto_progress(1)

import sys

sys.path.append("../src")
import aind_ccf_alignment_experiments.registration_methods as registration_methods
from aind_ccf_alignment_experiments.registration_methods import (
    get_sample_bounds,
    get_physical_size,
    compute_initial_translation,
    register_elastix,
    make_default_elx_parameter_object,
)

In [ ]:
SUBJECT_ID = 652506
SAMPLE_CHANNEL = "Ex_488_Em_525"
SAMPLE_LEVEL = 4  # data is scaled down by 2 ^ N in each direction or by 2 ^ 3N in total volume
SAMPLE_NAME = f"{SUBJECT_ID}_{SAMPLE_CHANNEL}"
SOURCE_IMAGE_INPUT_FILEPATH = (
    rf"data\input\{SUBJECT_ID}\{SAMPLE_NAME}_L{SAMPLE_LEVEL}.nii.gz"
)

# Also available at http://download.alleninstitute.org/informatics-archive/converted_mouse_ccf/average_template/
TARGET_IMAGE_INPUT_FILEPATH = (
    r"data\CCFv31/intensity/average_template_25.nii.gz"
)

BSPLINE_GRID_SPACING = 0.5  # isotropic B-spline control grid spacing in mm

RESULTS_PATH = f"data/results/{SUBJECT_ID}/BSPLINE/{BSPLINE_GRID_SPACING}mm_grid/2023.06.23/{SAMPLE_CHANNEL}"
REGISTERED_IMAGE_OUTPUT_FILEPATH = (
    f"{RESULTS_PATH}/{SAMPLE_NAME}_registered.nii.gz"
)

TRANSFORM_OUTPUT_FILEPATH = f"{RESULTS_PATH}/{SAMPLE_NAME}_transformresult.h5"

print(f"Registration results will be written to {RESULTS_PATH}")

## Load SmartSPIM Image

In [ ]:
source_image = itk.imread(SOURCE_IMAGE_INPUT_FILEPATH)
print(source_image)

## Load CCF Atlas Target Image

In [ ]:
target_image = itk.imread(TARGET_IMAGE_INPUT_FILEPATH, pixel_type=itk.F)

# Note: 3.1 template is in mm (3.0 was um)
print(target_image)

## Validate Data

We briefly evaluate the source and target images to ensure that image physical sizes are on the same order of magnitude as expected. A significant difference in image sizes could indicate a problem with image spacing.

A 3D spatial viewer such as ITKWidgets or Neuroglancer can be used to evaluate that source and target input images share a spatial orientation.

In [ ]:
print(f"CCF physical bounds: {get_sample_bounds(target_image)}")
print(f"SmartSPIM physical bounds: {get_sample_bounds(source_image)}")
print(f"CCF physical size: {get_physical_size(target_image)}")
print(f"SmartSPIM physical size: {get_physical_size(source_image)}")

## Initialize Registration with `itk`

We use tools available in the Insight Toolkit to align the source and target images so that they are initially overlapping in space.

In [ ]:
translation_transform, initialized_source_image = compute_initial_translation(
    source_image=source_image, target_image=target_image
)

In [ ]:
print(translation_transform)

In [ ]:
print(initialized_source_image)

In [ ]:
# Verify that the initialized source image bounds overlap with the target image

print(
    f"Original input source image bounds: {get_sample_bounds(source_image)[0]}, {get_sample_bounds(source_image)[1]}"
)
print(
    f"Translated source image bounds: {get_sample_bounds(initialized_source_image)[0]}, {get_sample_bounds(initialized_source_image)[1]}"
)
print(
    f"Target image bounds: {get_sample_bounds(target_image)[0]}, {get_sample_bounds(target_image)[1]}"
)

In [ ]:
# itkwidgets.compare_images(initialized_source_image, target_image)

## Register with `itk-elastix`

We use the tools developed in Elastix and made available via the ITKElastix Python module to perform multistage registration.

In [ ]:
parameter_object = make_default_elx_parameter_object()
print(parameter_object)

In [ ]:
bspline_map = parameter_object.GetParameterMap(2)
bspline_map["FinalGridSpacingInPhysicalUnits"] = (
    f"{BSPLINE_GRID_SPACING:0.6f}",
)
parameter_object.SetParameterMap(2, bspline_map)
print(parameter_object)

In [ ]:
(
    composite_transform,
    registered_source_image,
    registration_method,
) = register_elastix(
    source_image=initialized_source_image,
    target_image=target_image,
    parameter_object=parameter_object,
    verbose=True,
)

In [ ]:
# Verify that the registered source image bounds concide with the target image

print(
    f"Registered source image bounds: {get_sample_bounds(registered_source_image)[0]},"
    f"{get_sample_bounds(registered_source_image)[1]}"
)
print(
    f"Target image bounds: {get_sample_bounds(target_image)[0]}, {get_sample_bounds(target_image)[1]}"
)

In [ ]:
print(composite_transform)

In [ ]:
# itkwidgets.compare_images(registered_source_image, target_image)

## Save Outputs To Disk

Reproducible results should be saved to the capsule 'data' folder. Registration results from this notebook include:
- The registered, resampled SmartSPIM image. This can be compared with the target CCF average template image or CCF label atlas in a spatial viewer for visual evaluation of registration fitness.
- The sequence of transforms used to map from the source SmartSPIM sample space to target CCF space. We can map corresponding information in SmartSPIM source space such as segmentations or other markups into CCF space by applying this sequence of transformations.



In [ ]:
composite_transform.PrependTransform(translation_transform)

In [ ]:
os.makedirs(RESULTS_PATH, exist_ok=True)

itk.transformwrite(
    [composite_transform], TRANSFORM_OUTPUT_FILEPATH, compression=True
)

itk.imwrite(
    registered_source_image,
    REGISTERED_IMAGE_OUTPUT_FILEPATH,
    compression=True,
)

In [ ]:
# Write a summary file for reproducibility

with open(f"{RESULTS_PATH}/summary.txt", "w") as f:
    f.write(f"SUBJECT_ID {SUBJECT_ID}\n")
    f.write(f"SAMPLE_CHANNEL {SAMPLE_CHANNEL}\n")
    f.write(f"SAMPLE_LEVEL {SAMPLE_LEVEL}\n")
    f.write(f"SAMPLE_FILEPATH {SOURCE_IMAGE_INPUT_FILEPATH}\n")
    f.write(f"TARGET_FILEPATH {TARGET_IMAGE_INPUT_FILEPATH}\n")
    f.write("\n")
    f.write(f"INITIALIZATION_METHOD ITKCenteredVersorTransformInitializer\n")
    f.write(f"TRANSFORM_METHOD ELASTIX\n")
    f.write(str(parameter_object))
    f.write("\n")

    f.write(f"OUTPUT_TRANSFORM_FILEPATH {TRANSFORM_OUTPUT_FILEPATH}\n")
    f.write(f"OUTPUT_SAMPLE_FILEPATH {REGISTERED_IMAGE_OUTPUT_FILEPATH}\n")